In [ ]:
print("\n... IMPORTS STARTING ...\n")

print("\n... PIP/APT INSTALLS AND DOWNLOADS/ZIP STARTING ...")
!pip install -q pandarallel
!pip install -q tensorflow_model_optimization
!pip install -q --upgrade tensorflow_datasets
!pip install -q neural-structured-learning
print("... PIP/APT INSTALLS COMPLETE ...\n")

print("\n\tVERSION INFORMATION")
# Machine Learning and Data Science Imports
import tensorflow as tf; print(f"\t\t– TENSORFLOW VERSION: {tf.__version__}");
import tensorflow_addons as tfa; print(f"\t\t– TENSORFLOW ADDONS VERSION: {tfa.__version__}");
import pandas as pd; pd.options.mode.chained_assignment = None;
import numpy as np; print(f"\t\t– NUMPY VERSION: {np.__version__}");
import sklearn; print(f"\t\t– SKLEARN VERSION: {sklearn.__version__}");
from sklearn.preprocessing import RobustScaler, PolynomialFeatures
from pandarallel import pandarallel; pandarallel.initialize();
from sklearn.model_selection import GroupKFold;

# Built In Imports
from kaggle_datasets import KaggleDatasets
from collections import Counter
from datetime import datetime
from glob import glob
import warnings
import requests
import hashlib
import imageio
import IPython
import sklearn
import urllib
import zipfile
import pickle
import random
import shutil
import string
import json
import math
import time
import gzip
import ast
import sys
import io
import os
import gc
import re

# Visualization Imports
from matplotlib.colors import ListedColormap
import matplotlib.patches as patches
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm; tqdm.pandas();
import plotly.express as px
import seaborn as sns
from PIL import Image, ImageEnhance
import matplotlib; print(f"\t\t– MATPLOTLIB VERSION: {matplotlib.__version__}");
import plotly
import PIL
import cv2


def seed_it_all(seed=7):
    """ Attempt to be Reproducible """
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

    
print("\n\n... IMPORTS COMPLETE ...\n")
    
print("\n... EFFICIENTDET SETUP STARTING ...")

# SET LIBRARY DIRECTORY
LIB_DIR = "/kaggle/input/google-automl-efficientdetefficientnet-oct-2021"

# To give access to automl files
sys.path.insert(0, LIB_DIR)
sys.path.insert(0, os.path.join(LIB_DIR, "automl-master"))
sys.path.insert(0, os.path.join(LIB_DIR, "automl-master", "efficientdet"))
sys.path.insert(0, os.path.join(LIB_DIR, "automl-master", "efficientdet", "tf2"))
    

In [ ]:
# The name you gave to the TPU to use
TPU_WORKER = 'my-tpu-name'

# or you can also specify the grpc path directly
# TPU_WORKER = 'grpc://xxx.xxx.xxx.xxx:8470'

# The zone you chose when you created the TPU to use on GCP.
ZONE = 'us-east1-b'

# The name of the GCP project where you created the TPU to use on GCP.
PROJECT = 'my-tpu-project'

tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=TPU_WORKER, zone=ZONE, project=PROJECT)

In [ ]:
print(f"\n... ACCELERATOR SETUP STARTING ...\n")

# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    TPU = tf.distribute.cluster_resolver.TPUClusterResolver()  
except ValueError:
    TPU = None

if TPU:
    print(f"\n... RUNNING ON TPU - {TPU.master()}...")
    tf.config.experimental_connect_to_cluster(TPU)
    tf.tpu.experimental.initialize_tpu_system(TPU)
    strategy = tf.distribute.experimental.TPUStrategy(TPU)
else:
    print(f"\n... RUNNING ON CPU/GPU ...")
    # Yield the default distribution strategy in Tensorflow
    #   --> Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy() 

# What Is a Replica?
#    --> A single Cloud TPU device consists of FOUR chips, each of which has TWO TPU cores. 
#    --> Therefore, for efficient utilization of Cloud TPU, a program should make use of each of the EIGHT (4x2) cores. 
#    --> Each replica is essentially a copy of the training graph that is run on each core and 
#        trains a mini-batch containing 1/8th of the overall batch size
N_REPLICAS = strategy.num_replicas_in_sync
    
print(f"... # OF REPLICAS: {N_REPLICAS} ...\n")

print(f"\n... ACCELERATOR SETUP COMPLTED ...\n")

In [ ]:
print("\n... DATA ACCESS SETUP STARTED ...\n")

if TPU:
    # Google Cloud Dataset path to training and validation images
    DATA_DIR = KaggleDatasets().get_gcs_path('sartorius-cell-instance-segmentation')
    save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
else:
    # Local path to training and validation images
    DATA_DIR = "/kaggle/input/sartorius-cell-instance-segmentation"
    save_locally = None
    
print(f"\n... DATA DIRECTORY PATH IS:\n\t--> {DATA_DIR}")

print(f"\n... IMMEDIATE CONTENTS OF DATA DIRECTORY IS:")
for file in tf.io.gfile.glob(os.path.join(DATA_DIR, "*")): print(f"\t--> {file}")

    
print("\n\n... DATA ACCESS SETUP COMPLETED ...\n")

In [ ]:
print(f"\n... XLA OPTIMIZATIONS STARTING ...\n")

print(f"\n... CONFIGURE JIT (JUST IN TIME) COMPILATION ...\n")
# enable XLA optmizations (10% speedup when using @tf.function calls)
tf.config.optimizer.set_jit(True)

print(f"\n... XLA OPTIMIZATIONS COMPLETED ...\n")

## تعريفات البيانات الأساسية وتهيئتها

In [ ]:
print("\n... BASIC DATA SETUP STARTING ...\n\n")

print("\n... SET PATH INFORMATION ..\n")
SEG_DIR = "/kaggle/input/sartorius-segmentation-train-mask-dataset-npz"
LC_DIR = os.path.join(DATA_DIR, "LIVECell_dataset_2021")
LC_ANN_DIR = os.path.join(LC_DIR, "annotations")
LC_IMG_DIR = os.path.join(LC_DIR, "images")
TRAIN_DIR = os.path.join(DATA_DIR, "train")
TEST_DIR = os.path.join(DATA_DIR, "test")
SEMI_DIR = os.path.join(DATA_DIR, "train_semi_supervised")

print("\n... TRAIN DATAFRAME ...\n")

# FIX THE TRAIN DATAFRAME (GROUP THE RLEs TOGETHER)
TRAIN_CSV = os.path.join(DATA_DIR, "train.csv")
train_df = pd.read_csv(TRAIN_CSV)
display(train_df)

print("\n... SS DATAFRAME ..\n")
SS_CSV = os.path.join(DATA_DIR, "sample_submission.csv")
ss_df = pd.read_csv(SS_CSV)
ss_df["img_path"] = ss_df["id"].apply(lambda x: os.path.join(TEST_DIR, x+".png")) # Capture Image Path As Well
display(ss_df)

CELL_TYPES = list(train_df.cell_type.unique())
FIRST_SHSY5Y_IDX = 0
FIRST_ASTRO_IDX  = 1
FIRST_CORT_IDX   = 2

# This is required for plotting so that the smaller distributions get plotted on top
ARB_SORT_MAP = {"astro":0, "shsy5y":1, "cort":2}

print("\n... CELL TYPES ..")
for x in CELL_TYPES: print(f"\t--> {x}")
    
print("\n\n... BASIC DATA SETUP FINISHING ...\n")

### وظائف وفصول المساعد

In [ ]:
# ref: https://www.kaggle.com/paulorzp/run-length-encode-and-decode
# modified from: https://www.kaggle.com/inversion/run-length-decoding-quick-start
def rle_decode(mask_rle, shape, color=1):
    """ TBD
    
    Args:
        mask_rle (str): run-length as string formated (start length)
        shape (tuple of ints): (height,width) of array to return 
    
    Returns: 
        Mask (np.array)
            - 1 indicating mask
            - 0 indicating background

    """
    # Split the string by space, then convert it into a integer array
    s = np.array(mask_rle.split(), dtype=int)

    # Every even value is the start, every odd value is the "run" length
    starts = s[0::2] - 1
    lengths = s[1::2]
    ends = starts + lengths

    # The image image is actually flattened since RLE is a 1D "run"
    if len(shape)==3:
        h, w, d = shape
        img = np.zeros((h * w, d), dtype=np.float32)
    else:
        h, w = shape
        img = np.zeros((h * w,), dtype=np.float32)

    # The color here is actually just any integer you want!
    for lo, hi in zip(starts, ends):
        img[lo : hi] = color
        
    # Don't forget to change the image back to the original shape
    return img.reshape(shape)

# https://www.kaggle.com/namgalielei/which-reshape-is-used-in-rle
def rle_decode_top_to_bot_first(mask_rle, shape):
    """ TBD
    
    Args:
        mask_rle (str): run-length as string formated (start length)
        shape (tuple of ints): (height,width) of array to return 
    
    Returns:
        Mask (np.array)
            - 1 indicating mask
            - 0 indicating background

    """
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape((shape[1], shape[0]), order='F').T  # Reshape from top -> bottom first

# ref.: https://www.kaggle.com/stainsby/fast-tested-rle
def rle_encode(img):
    """ TBD
    
    Args:
        img (np.array): 
            - 1 indicating mask
            - 0 indicating background
    
    Returns: 
        run length as string formated
    """
    
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)


def flatten_l_o_l(nested_list):
    """ Flatten a list of lists """
    return [item for sublist in nested_list for item in sublist]


def load_json_to_dict(json_path):
    """ tbd """
    with open(json_path) as json_file:
        data = json.load(json_file)
    return data

# https://github.com/PyImageSearch/imutils/blob/master/imutils/convenience.py
def grab_contours(cnts):
    """ TBD """
    
    # if the length the contours tuple returned by cv2.findContours
    # is '2' then we are using either OpenCV v2.4, v4-beta, or
    # v4-official
    if len(cnts) == 2:
        cnts = cnts[0]

    # if the length of the contours tuple is '3' then we are using
    # either OpenCV v3, v4-pre, or v4-alpha
    elif len(cnts) == 3:
        cnts = cnts[1]

    # otherwise OpenCV has changed their cv2.findContours return
    # signature yet again and I have no idea WTH is going on
    else:
        raise Exception(("Contours tuple must have length 2 or 3, "
            "otherwise OpenCV changed their cv2.findContours return "
            "signature yet again. Refer to OpenCV's documentation "
            "in that case"))

    # return the actual contours array
    return cnts

def get_contour_bbox(msk):
    """ Function to return the bounding box (tl, br) for a given mask """
    
    # Get contour(s) --> There should be only one
    cnts = cv2.findContours(msk.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    contour = grab_contours(cnts)
    
    if len(contour)==0:
        return None
    else:
        contour = contour[0]
    
    # Get extreme coordinates
    tl = (tuple(contour[contour[:, :, 0].argmin()][0])[0], 
          tuple(contour[contour[:, :, 1].argmin()][0])[1])
    br = (tuple(contour[contour[:, :, 0].argmax()][0])[0], 
          tuple(contour[contour[:, :, 1].argmax()][0])[1])
    return tl, br

def tf_load_png(img_path):
    return tf.image.decode_png(tf.io.read_file(img_path), channels=3)

def get_img_and_mask(img_path, annotation, width, height, mask_only=False, rle_fn=rle_decode):
    """ Capture the relevant image array as well as the image mask """
    img_mask = np.zeros((height, width), dtype=np.uint8)
    for i, annot in enumerate(annotation): 
        img_mask = np.where(rle_fn(annot, (height, width))!=0, i, img_mask)
    
    # Early Exit
    if mask_only:
        return img_mask
    
    # Else Return images
    img = tf_load_png(img_path)[..., 0]
    return img, img_mask

def plot_img_and_mask(img, mask, bboxes=None, invert_img=True, boost_contrast=True):
    """ Function to take an image and the corresponding mask and plot
    
    Args:
        img (np.arr): 1 channel np arr representing the image of cellular structures
        mask (np.arr): 1 channel np arr representing the instance masks (incrementing by one)
        bboxes (list of tuples, optional): (tl, br) coordinates of enclosing bboxes
        invert_img (bool, optional): Whether or not to invert the base image
        boost_contrast (bool, optional): Whether or not to boost contrast of the base image
        
    Returns:
        None; Plots the two arrays and overlays them to create a merged image
    """
    plt.figure(figsize=(20,10))
    
    plt.subplot(1,3,1)
    _img = np.tile(np.expand_dims(img, axis=-1), 3)
    
    # Flip black-->white ... white-->black
    if invert_img:
        _img = _img.max()-_img
    
    if boost_contrast:
        _img = np.asarray(ImageEnhance.Contrast(Image.fromarray(_img)).enhance(16))
    
    if bboxes:
        for i, bbox in enumerate(bboxes):
            mask = cv2.rectangle(mask, bbox[0], bbox[1], (i+1, 0, 0), thickness=2)
    
    plt.imshow(_img)
    plt.axis(False)
    plt.title("Cell Image", fontweight="bold")
    
    plt.subplot(1,3,2)
    _mask = np.zeros_like(_img)
    _mask[..., 0] = mask
    plt.imshow(mask, cmap="inferno")
    plt.axis(False)
    plt.title("Instance Segmentation Mask", fontweight="bold")
    
    merged = cv2.addWeighted(_img, 0.75, np.clip(_mask, 0, 1)*255, 0.25, 0.0,)
    plt.subplot(1,3,3)
    plt.imshow(merged)
    plt.axis(False)
    plt.title("Cell Image w/ Instance Segmentation Mask Overlay", fontweight="bold")
    
    plt.tight_layout()
    plt.show()
    
def pd_get_bboxes(row):
    """ Get all bboxes for a given row/cell-image """
    mask = get_img_and_mask(row.img_path, row.annotation, row.width, row.height, mask_only=True)
    return [get_contour_bbox(np.where(mask==i, 1, 0).astype(np.uint8)) for i in range(1, mask.max()+1)]

def get_bbox_stats(bbox_list, style="area"): 
    """ TBD 
    
    Args:
        bbox_list(): TBD
        style (str, optional): TBD
    Returns:
        TBD
        """
    bbox_stats = []
    for box in bbox_list:
        try:
            if style=="area":
                bbox_stats.append(float((box[1][0]-box[0][0])*(box[1][1]-box[0][1])))
            elif style=="width":
                bbox_stats.append(float(box[1][0]-box[0][0]))
            else:
                bbox_stats.append(float(box[1][1]-box[0][1]))
        except:
            bbox_stats.append(0.0)
    return bbox_stats

## إنشاء واستكشاف قواعد البيانات

`# قم بتحديث إطار بيانات القطار `

In [ ]:
# Aggregate under training 
train_df["img_path"] = train_df["id"].apply(lambda x: os.path.join(TRAIN_DIR, x+".png")) # Capture Image Path As Well
tmp_df = train_df.drop_duplicates(subset=["id", "img_path"]).reset_index(drop=True)
tmp_df["annotation"] = train_df.groupby("id")["annotation"].agg(list).reset_index(drop=True)
train_df = tmp_df.copy()
train_df["seg_path"] = train_df.id.apply(lambda x: os.path.join(SEG_DIR, f"{x}.npz"))
display(train_df)

## تصور بيانات القطار

In [ ]:
for i in range(2, 70, 8):
    print(f"\n\n\n\n... RELEVANT DATAFRAME ROW - INDEX={i} ...\n")
    display(train_df.iloc[i:i+1])
    img, msk = get_img_and_mask(**train_df[["img_path", "annotation", "width", "height"]].iloc[i].to_dict())
    plot_img_and_mask(img, msk)

In [ ]:
x1 = rle_decode_top_to_bot_first(train_df.iloc[0].annotation[0], (train_df.iloc[0].height, train_df.iloc[0].width))
x2 = rle_decode(train_df.iloc[0].annotation[0], (train_df.iloc[0].height, train_df.iloc[0].width))

plt.figure(figsize=(15,6))
plt.subplot(1,2,1)
plt.imshow(x1, cmap="inferno")
plt.axis(False)
plt.title("NamGalielei RLE Decode Function", fontweight="bold")
plt.subplot(1,2,2)
plt.imshow(x2, cmap="inferno")
plt.axis(False)
plt.title("Original RLE Decode Function", fontweight="bold")
plt.tight_layout()
plt.show()
print(f"\n... THERE ARE {(x1!=x2).sum()} PIXELS IN DISAGREEMENT WHEN USING THE TWO FUNCTIONS ON A SINGLE CELL...\n")

img1, msk1 = get_img_and_mask(**train_df[["img_path", "annotation", "width", "height"]].iloc[0].to_dict())
img2, msk2 = get_img_and_mask(**train_df[["img_path", "annotation", "width", "height"]].iloc[0].to_dict(), rle_fn=rle_decode_top_to_bot_first)

plot_img_and_mask(img1, msk1)
plot_img_and_mask(img2, msk2)

print(f"\n... THERE ARE {(msk2!=msk1).sum()} PIXELS IN DISAGREEMENT WHEN USING THE TWO FUNCTIONS ON ALL CELL MASK ...\n")

In [ ]:
print("\n\n... WIDTH VALUE COUNTS ...")
for k,v in train_df.width.value_counts().items():
    print(f"\t--> There are {v} images with WIDTH={k}")

print("\n\n... HEIGHT VALUE COUNTS ...")
for k,v in train_df.height.value_counts().items():
    print(f"\t--> There are {v} images with HEIGHT={k}")

print("\n\n... AREA COUNTS ...")
for k,v in (train_df.width*train_df.height).value_counts().items():
    print(f"\t--> There are {v} images with AREA={k}")

print("\n\n... NOTE: ALL THE IMAGES ARE THE SAME SIZE ...\n")

print("\n\n... PLATE TIME VALUE COUNTS ...")
for k,v in train_df.plate_time.value_counts().items():
    print(f"\t--> There are {v} images with PLATE_TIME={k}")
fig = px.histogram(train_df, x="plate_time", color="cell_type", title="<b>Plate Time Histogram</b>")
fig.show()

print("\n\n... SAMPLE DATE VALUE COUNTS ...")
for k,v in train_df.sample_date.value_counts().items():
    print(f"\t--> There are {v} images with SAMPLE_DATE={k}")
fig = px.histogram(train_df, train_df.sample_date.apply(lambda x: x.replace("-", "_")), color="cell_type", title="<b>Sample Date Value Histogram</b>")
fig.show()

print("\n\n... ELAPSED TIME DELTA VALUE COUNTS ...")
for k,v in train_df.elapsed_timedelta.value_counts().items():
    print(f"\t--> There are {v} images with SAMPLE_DATE={k}")
fig = px.histogram(train_df, "elapsed_timedelta", color="cell_type", title="<b>Elapsed Time Delta Value Histogram</b>")
fig.show()
    
print("\n\n... SAMPLE ID VALUE COUNTS (>1) ...")
print(f"\t--> There are {len(train_df[train_df.sample_id.isin([x for x,v in train_df.sample_id.value_counts().items() if v>1])])} SAMPLE_IDs with more than one image\n")
for k,v in train_df[train_df.sample_id.isin([x for x,v in train_df.sample_id.value_counts().items() if v>1])].reset_index()["sample_id"].value_counts().items():
    print(f"\t--> There are {v} images with SAMPLE_ID={k}")
fig = px.histogram(train_df[train_df.sample_id.isin([x for x,v in train_df.sample_id.value_counts().items() if v>1])].reset_index(), "sample_id", color="cell_type", title="<b>Sample ID Value Histogram</b>")
fig.show()

    
print("\n\n... CELL TYPE VALUE COUNTS ...")
for k,v in train_df.cell_type.value_counts().items():
    print(f"\t--> There are {v} images with CELL_TYPE={k}")
    
fig = px.histogram(train_df, x="cell_type", title="<b>Cell Type Histogram</b>")
fig.show()

for ct in CELL_TYPES:
    print(f"\n\n... SHOWING THREE EXAMPLES OF CELL TYPE {ct.upper()} ...\n")
    for i in range(3):
        img, msk = get_img_and_mask(**train_df[train_df.cell_type==ct][["img_path", "annotation", "width", "height"]].sample(3).reset_index(drop=True).iloc[i].to_dict())
        plot_img_and_mask(img, msk)

In [ ]:
DEFER = True

if not DEFER:
    LC_CELL_TYPES = os.listdir(os.path.join(LC_ANN_DIR, "LIVECell_single_cells"))

    print("\n... LOADING TRAIN COCO JSON ...\n")
    LC_COCO_TRAIN = os.path.join(LC_ANN_DIR, "LIVECell", "livecell_coco_train.json")

    print("\n... LOADING VALIDATION COCO JSON ...\n")
    LC_COCO_VAL = os.path.join(LC_ANN_DIR, "LIVECell", "livecell_coco_val.json")

    print("\n... LOADING TEST COCO JSON ...\n")
    LC_COCO_TEST = os.path.join(LC_ANN_DIR, "LIVECell", "livecell_coco_test.json")

    LC_SC_TRAIN = {
        lc_ct:os.path.join(LC_ANN_DIR, "LIVECell_single_cells", lc_ct, f"livecell_{lc_ct}_train.json") \
        for lc_ct in LC_CELL_TYPES
    }
    LC_SC_VAL = {
        lc_ct:os.path.join(LC_ANN_DIR, "LIVECell_single_cells", lc_ct, f"livecell_{lc_ct}_val.json") \
        for lc_ct in LC_CELL_TYPES
    }
    LC_SC_TEST = {
        lc_ct:os.path.join(LC_ANN_DIR, "LIVECell_single_cells", lc_ct, f"livecell_{lc_ct}_test.json") \
        for lc_ct in LC_CELL_TYPES
    }

    print(LC_SC_TRAIN)
    print(LC_SC_VAL)
    print(LC_SC_TEST)

In [ ]:
semi_df = pd.DataFrame()

semi_df["cell_type"] = [x.split("[", 1)[0] for x in tf.io.gfile.listdir(SEMI_DIR)]
semi_df["compound"] = [x.split("]", 1)[0].split("[", 1)[-1] for x in tf.io.gfile.listdir(SEMI_DIR)]
semi_df["img_path"] = tf.io.gfile.glob(os.path.join(SEMI_DIR, "**"))

fig = px.histogram(semi_df, "cell_type", color="compound")
fig.show()

fig = px.histogram(semi_df, "compound", color="cell_type")
fig.show()

In [ ]:
plt.figure(figsize=(20,26))
for i, img_path in zip(range(15), semi_df.img_path.to_list()):
    plt.subplot(5,3,i+1)
    plt.imshow((255-np.asarray(ImageEnhance.Contrast(Image.fromarray(tf_load_png(img_path).numpy())).enhance(16))), cmap="inferno")
    plt.axis(False)
    plt.title(img_path.rsplit("/", 1)[-1].rsplit(".", 1)[0], fontweight="bold")
    
plt.tight_layout()
plt.show()

In [ ]:
DEMO_IDX = 11
img, msk = get_img_and_mask(**train_df[["img_path", "annotation", "width", "height"]].iloc[DEMO_IDX].to_dict())
plot_img_and_mask(img, msk)

plt.figure(figsize=(20, min(80, msk.max()//2)))
for i in range(1, msk.max()+1):
    plt.subplot(10,10,i)
    tl, br = get_contour_bbox(np.where(msk==i, 1, 0).astype(np.uint8))
    plt.imshow(np.asarray(ImageEnhance.Contrast(Image.fromarray(255-img.numpy())).enhance(16))[tl[1]:br[1], tl[0]:br[0]], cmap="magma")
    plt.axis(False)
    plt.title(f"{i}", fontweight="bold")
    if i==100:
        break
    
plt.tight_layout()
plt.show()

In [ ]:
# Takes about 1 minute
print("\n... CREATE FULL SCALE BBOXES ...\n")
train_df["bboxes"] = train_df.parallel_apply(pd_get_bboxes, axis=1)
display(train_df.head())

print("\n... CREATE SCALED DOWN (0-1) BBOXES ...\n")
IMG_O_W, IMG_O_H = train_df.iloc[0].width, train_df.iloc[0].height
train_df["scaled_bboxes"] = train_df.bboxes.progress_apply(lambda box_list: [((box[0][0]/IMG_O_W, box[0][1]/IMG_O_H), (box[1][0]/IMG_O_W,box[1][1]/IMG_O_H)) if box else None for box in box_list])

# CORT
img, msk = get_img_and_mask(**train_df[["img_path", "annotation", "width", "height"]].iloc[FIRST_SHSY5Y_IDX].to_dict())
plot_img_and_mask(img, msk, bboxes=train_df.iloc[FIRST_SHSY5Y_IDX].bboxes)

# ASTRO
img, msk = get_img_and_mask(**train_df[["img_path", "annotation", "width", "height"]].iloc[FIRST_ASTRO_IDX].to_dict())
plot_img_and_mask(img, msk, bboxes=train_df.iloc[FIRST_ASTRO_IDX].bboxes)

# SHSY5Y
img, msk = get_img_and_mask(**train_df[["img_path", "annotation", "width", "height"]].iloc[FIRST_CORT_IDX].to_dict())
plot_img_and_mask(img, msk, bboxes=train_df.iloc[FIRST_CORT_IDX].bboxes)

In [ ]:
train_df["bbox_widths"] = train_df.bboxes.apply(lambda x: get_bbox_stats(x, style="width"))
train_df["bbox_heights"] = train_df.bboxes.apply(lambda x: get_bbox_stats(x, style="height"))
train_df["bbox_areas"] = train_df.bboxes.apply(lambda x: get_bbox_stats(x, style="area"))

train_df["scaled_bbox_widths"] = train_df.scaled_bboxes.apply(lambda x: get_bbox_stats(x, style="width"))
train_df["scaled_bbox_heights"] = train_df.scaled_bboxes.apply(lambda x: get_bbox_stats(x, style="height"))
train_df["scaled_bbox_areas"] = train_df.scaled_bboxes.apply(lambda x: get_bbox_stats(x, style="area"))

display(train_df.head())

# Plot
px.scatter(train_df.sort_values(by="cell_type", key=lambda x: x.map(ARB_SORT_MAP))[["cell_type", "bbox_widths", "bbox_heights", "bbox_areas"]].explode(column=["bbox_widths","bbox_heights", "bbox_areas"]), x="bbox_widths", y="bbox_heights", color="cell_type", title="<b>Cell Bounding Box Sizes (WxH)</b>")

## Trining UNet

In [ ]:
from pathlib import Path
import cv2

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers.experimental import preprocessing

from IPython.display import clear_output
import matplotlib.pyplot as plt

In [ ]:
# paths
# input
DIR = '../input/sartorius-cell-instance-segmentation'
train_csv = os.path.join(DIR,'train.csv') 
train_path =  os.path.join(DIR, 'train/')
test_path = os.path.join(DIR, 'test/')

# output 
csv_output = os.path.join('./', 'submission.csv') 
model_output = os.path.join('./', 'unet_keras_model.h5')

In [ ]:
def rle_decode(mask_rle, shape, color=1):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background.
    ref: https://www.kaggle.com/inversion/run-length-decoding-quick-start
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros((shape[0] * shape[1]), dtype=np.float32)
    for lo, hi in zip(starts, ends):
        img[lo : hi] = color
    return img.reshape(shape)


def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    ref: https://www.kaggle.com/dragonzhang/positive-score-with-detectron-3-3-inference
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)


def get_mask(image_id, df):
    '''
    Uses rle_decode() to get ndarray from mask using image_id in dataframe (df).
    ref: https://www.kaggle.com/barteksadlej123/sartors-tf-starter
    '''
    current = df[df["id"] == image_id]
    labels = current["annotation"].tolist()
    
    mask = np.zeros((HEIGHT, WIDTH))
    for label in labels:
        mask += rle_decode(label, (HEIGHT, WIDTH))
    mask = mask.clip(0, 1)
    
    return mask


#  fix overlaps: 

def check_overlap(msk):
    '''
    Checks if there are overlap in a mask (msk).
    ref: https://www.kaggle.com/awsaf49/sartorius-fix-overlap
    '''
    msk = msk.astype(np.bool).astype(np.uint8)
    return np.any(np.sum(msk, axis=-1)>1)


def fix_overlap(msk):
    '''
    Args:
        mask: multi-channel mask, each channel is an instance of cell, shape:(520,704,None)
    Returns:
        multi-channel mask with non-overlapping values, shape:(520,704,None) 
    ref: https://www.kaggle.com/awsaf49/sartorius-fix-overlap
    '''
    msk = np.array(msk)
    msk = np.pad(msk, [[0,0],[0,0],[1,0]])
    ins_len = msk.shape[-1]
    msk = np.argmax(msk,axis=-1)
    msk = tf.keras.utils.to_categorical(msk, num_classes=ins_len)
    msk = msk[...,1:]
    msk = msk[...,np.any(msk, axis=(0,1))]
    return msk

In [ ]:

# make predictions for test set: 

def make_predictions(dataset, num, keras_model):
    '''
    For a tf.Dataset, makes predictions for n=num (num =-1 or all_images takes all images in the dataset), 
    images using a keras_model. Returns a list of predicted masks, each as ndarray. 
    '''
    predictions = []
    if dataset:
        for image in dataset.take(num):
            image = image[None]
            pred_mask = keras_model.predict(image)
            # changes shape from (1,512,512,1) to (512,512)
            pred_mask = pred_mask[0, :, :, 0]
            # fix overlaps
            if check_overlap(msk=pred_mask)==True:
                pred_mask = pred_mask[None]
                pred_mask = fix_overlap(msk=pred_mask)
            # transforms ndarray values to 0s and 1s
            pred_mask =  np.where( pred_mask > 0.5, 1, 0)
            predictions.append(pred_mask)
    return predictions

In [ ]:
# constants

DEBUG = False

SEED = 123
WIDTH, HEIGHT = 704, 520
RESIZE_WIDTH, RESIZE_HEIGHT = 512, 512
BATCH_SIZE = 32
BUFFER_SIZE = 32

VAL_SPLIT = 0.2

AUTO = tf.data.AUTOTUNE

EPOCHS = 2

In [ ]:
# train and validation split
train = pd.read_csv(train_csv)
train.head()

n_ids = train.id.nunique()

if DEBUG:
    unique_ids_train = list(set(train['id'].tolist()))[:BATCH_SIZE]
    unique_ids_valid = list(set(train['id'].tolist()))[BATCH_SIZE:2*BATCH_SIZE]
else:
    unique_ids_train = list(set(train['id'].tolist()))[:int(n_ids * (1 - VAL_SPLIT))]
    unique_ids_valid = list(set(train['id'].tolist()))[int(n_ids * (1 - VAL_SPLIT)):]


temp = pd.DataFrame()
for sample_id in unique_ids_train:
    query = train[train.id == sample_id]
    temp = pd.concat([temp, query])
train = temp
train = train.reset_index(drop=True)

temp = pd.DataFrame()
for sample_id in unique_ids_valid:
    query = train[train.id == sample_id]
    temp = pd.concat([temp, query])
valid = temp
valid = train.reset_index(drop=True)
    
TRAIN_LENGTH = train['id'].nunique()
STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE

VALID_LENGTH = valid['id'].nunique()
VALIDATION_STEPS = VALID_LENGTH // BATCH_SIZE
# training data generator 
def train_generator(df):
    image_ids = set(df['id'].tolist())
    
    for image_id in image_ids:
        image = cv2.imread(os.path.join(train_path, image_id) + '.png') 
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        mask = get_mask(image_id, df)
        
        image = cv2.resize(image, (RESIZE_HEIGHT, RESIZE_WIDTH))
        mask = cv2.resize(mask, (RESIZE_HEIGHT, RESIZE_WIDTH))
        mask = mask.reshape((*mask.shape, 1))
        
        image = image.astype(np.float32)
        mask = mask.astype(np.int32)
        
        yield image, mask

In [ ]:
# use the generator to get training and validation sets
train_ds = tf.data.Dataset.from_generator(
    lambda : train_generator(train), 
    output_types=(tf.float32, tf.int32),
    output_shapes=((RESIZE_HEIGHT, RESIZE_WIDTH, 3), (RESIZE_HEIGHT, RESIZE_WIDTH, 1)))

valid_ds = tf.data.Dataset.from_generator(
    lambda : train_generator(valid), 
    output_types=(tf.float32, tf.int32),
    output_shapes=((RESIZE_HEIGHT, RESIZE_WIDTH, 3), (RESIZE_HEIGHT, RESIZE_WIDTH, 1)))

In [ ]:
# "the following class performs a simple augmentation by randomly-flipping an image"
class Augment(tf.keras.layers.Layer):
    def __init__(self, seed=SEED):
        super().__init__()
        
        self.augment_inputs = preprocessing.RandomFlip('horizontal', seed=seed)
        self.augment_labels = preprocessing.RandomFlip('horizontal', seed=seed)
        
    def call(self, inputs, labels):
        inputs = self.augment_inputs(inputs)
        labels = self.augment_labels(labels)
        return inputs, labels

In [ ]:
# "build the input pipeline, applying the augmentation after batching the inputs"

train_ds = (
    train_ds
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE)
    .repeat()
    .map(Augment())
    .prefetch(AUTO))

valid_ds = (
    valid_ds
    .batch(BATCH_SIZE)
    .repeat()
    .prefetch(AUTO))

In [ ]:
# "visualize an image example and its corresponding mask from the dataset"

def display(display_list):
    plt.figure(figsize=(20, 20))

    title = ['Input Image', 'True Mask', 'Predicted Mask']

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        plt.axis('off')
    plt.show()
    
for images, masks in train_ds.take(2):
    sample_image, sample_mask = images[0], masks[0]
    display([sample_image, sample_mask])

In [ ]:
# base model and encoder (down_stack)

base_model = tf.keras.applications.MobileNetV2(input_shape=[RESIZE_HEIGHT, RESIZE_WIDTH, 3], include_top=False)

# use the activations of these layers
layer_names = [
    'block_1_expand_relu',   # 64x64
    'block_3_expand_relu',   # 32x32
    'block_6_expand_relu',   # 16x16
    'block_13_expand_relu',  # 8x8
    'block_16_project',      # 4x4
]

base_model_outputs = [base_model.get_layer(name).output for name in layer_names]

# create the feature extraction model
down_stack = tf.keras.Model(inputs=base_model.input, outputs=base_model_outputs)
down_stack.trainable = False

In [ ]:
# decoder / upsampler

def upsample(filters, size, apply_dropout=False):
    '''
    filters: Integer, the dimensionality of the output space (i.e. the number of output filters in the convolution). 
    size: An integer or tuple/list of 2 integers, specifying the height and width of the 2D convolution window. 
    Can be a single integer to specify the same value for all spatial dimensions. 
    return: 4+D tensor with shape: batch_shape + (channels, rows, cols) if data_format='channels_first' 
    or 4+D tensor with shape: batch_shape + (rows, cols, channels) if data_format='channels_last
    ref: https://www.tensorflow.org/tutorials/generative/pix2pix?hl=nb
    '''
    initializer = tf.random_normal_initializer(0., 0.02)
    result = tf.keras.Sequential()
    result.add(
    tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                    padding='same',
                                    kernel_initializer=initializer,
                                    use_bias=False))
    result.add(tf.keras.layers.BatchNormalization())
    if apply_dropout:
        result.add(tf.keras.layers.Dropout(0.5))
    result.add(tf.keras.layers.ReLU())
    return result


up_stack = [
    upsample(512, 3),  # 4x4 -> 8x8
    upsample(256, 3),  # 8x8 -> 16x16
    upsample(128, 3),  # 16x16 -> 32x32
    upsample(64, 3),   # 32x32 -> 64x64
]

In [ ]:
# U-Net model 
def unet_model(output_channels : int):
    inputs = tf.keras.layers.Input(shape=[RESIZE_HEIGHT , RESIZE_WIDTH, 3])
    # "downsampling through the model"
    skips = down_stack(inputs)
    x = skips[-1]
    skips = reversed(skips[:-1])
    # "sampling and establishing the skip connections"
    for up, skip in zip(up_stack, skips):
        x = up(x)
        concat = tf.keras.layers.Concatenate()
        x = concat([x, skip])
    # "this is the last layer of the model"
    last = tf.keras.layers.Conv2DTranspose(
        filters=output_channels, kernel_size=3, strides=2,
        padding='same', activation='sigmoid') #64x64 -> 128x128
    
    x = last(x)
    
    return tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
# compile model 
## in a segmentation task each pixel is given a class 
## OUTPUT_CLASSES: number of classes that can be assigned to each pixel 

OUTPUT_CLASSES = 1

model = unet_model(output_channels=OUTPUT_CLASSES)

opt = keras.optimizers.Adam(learning_rate=0.01)

model.compile(optimizer=opt,
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [ ]:
# visualize model architecture 
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
# try out the model to check what it predicts before training

def create_mask(pred_mask):
    pred_mask = tf.where(pred_mask > 0.5,1,0)
    return pred_mask


def show_predictions(dataset=None, num=1):
    if dataset:
        for image, mask in dataset.take(num):
            pred_mask = model.predict(image)
            display([image[0], mask[0], create_mask(pred_mask[1])])
    else:
        display([sample_image, sample_mask,
                 create_mask(model.predict(sample_image[tf.newaxis, ...])[0])])

        
show_predictions(train_ds)

In [ ]:
# training

# "observe how the model improves while it is training"
class DisplayCallback(tf.keras.callbacks.Callback):
    def __init__(self):
        super().__init__()
    def on_epoch_end(self, epoch, logs=None):
        clear_output(wait=False)
        show_predictions()
        print ('\nSample Prediction after epoch {}\n'.format(epoch+1))
# display callback defined above
display_cb = DisplayCallback()

# "save the Keras model or model weights at some frequency"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    model_output,
    save_best_only=True,
    save_weights_only=True,
)

# "reduce learning rate when a metric has stopped improving"
# documentation: https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ReduceLROnPlateau
lr_reduce = tf.keras.callbacks.ReduceLROnPlateau( monitor='val_loss', factor=0.1, patience=10, verbose=0,
    mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)

model_history = model.fit(train_ds, epochs=EPOCHS,
                          steps_per_epoch=STEPS_PER_EPOCH,
                          validation_steps=VALIDATION_STEPS,
                          validation_data=valid_ds,
                          callbacks=[display_cb, model_checkpoint, lr_reduce])

In [ ]:
# plot training curve
loss = model_history.history['loss']
val_loss = model_history.history['val_loss']
plt.figure()
plt.plot(model_history.epoch, loss, 'r', label='Training loss')
plt.plot(model_history.epoch, val_loss, 'bo', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss Value')
plt.ylim([0, 1])
plt.legend()
plt.show()

In [ ]:
# test data generator 
test_ids = [  os.path.join(test_path, each)  for each in os.listdir(test_path) if each.endswith('.png')]
def test_generator(image_ids):
    for image_id in image_ids:
        image = cv2.imread(image_id) 
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)        
        image = cv2.resize(image, (RESIZE_HEIGHT, RESIZE_WIDTH))
        image = image.astype(np.float32)
        yield image

In [ ]:
# test dataset from test data generator 
test_ds = tf.data.Dataset.from_generator(
    lambda : test_generator(test_ids), 
    output_types=(tf.float32),
    output_shapes=((RESIZE_HEIGHT, RESIZE_WIDTH, 3)) )

In [ ]:
# test image ids and predictions
test_predictions = make_predictions(dataset=test_ds, num=len(test_ids), keras_model=model)

In [ ]:
# encode predections in the RL format
test_predictions = [rle_encode(mask) for mask in test_predictions] 

In [ ]:
# transform full image paths to ids 
test_ids = [Path(ID).stem for ID in test_ids]

In [ ]:
# generate submission data frame 
submisssion = pd.DataFrame.from_dict({'id': test_ids, 'predicted': test_predictions} )
submisssion = submisssion.sort_values( ['id'], ascending=True )
print(submisssion.head(), 'n')
submisssion.to_csv(csv_output, index=False)